In [2]:
%run imports.ipynb
import allel
import zarr
import pandas as pd
import petl as etl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
palette = sns.color_palette()

Workflow:


    - Split 3R Free region in 2: To 2.82Mb, from 2.92Mb

In [3]:
calldata_out= zarr.open('data/outgroup_alleles_phase2.zarr/')
calldata_biallel= zarr.open('data/phase2_biallel_allele_count.zarr/')

In [4]:
an_col = 'AOcol', 'BFcol', 'GNcol', 'CIcol', 'GHcol'
an_gam = 'GHgam', 'CMgam', 'BFgam', 'GNgam', 'GQgam', 'UGgam', 'GAgam', 'FRgam'
an_na = 'GM', 'GW'
out_species = ['arab', 'chri', 'epir', 'mela', 'meru', 'quad']

------------------------------
## Taking out the first region

In [2]:
def count_alleles_pos(chrom, start, stop):
    
    metadata = pd.read_csv("samples.meta.txt", sep="\t")
    pos_all = allel.SortedIndex(callset_biallel[chrom]["variants/POS"])
    loc = pos_all.intersect_range(start, stop)
    loc1, loc2 = pos_all.locate_intersection(loc)
    geno = allel.GenotypeChunkedArray(callset_biallel[chrom]["calldata/GT"])
    geno_subset = geno.subset(sel0=loc1)
    ac = geno_subset.count_alleles()[:]
    grp = metadata.groupby("population")
    grp_indices = grp.indices
    ac_subpops = geno_subset.count_alleles_subpops(grp_indices)

    return ac_subpops

In [3]:
pos_all = allel.SortedIndex(callset_biallel['3R']["variants/POS"])
loc = pos_all.intersect_range(1000000, 2820000)
loc1, loc2 = pos_all.locate_intersection(loc)

In [4]:
ac_3R_first = count_alleles_pos('3R', 1000000, 2820000)
ac_3R_first

<AlleleCountsChunkedTable shape=(522469,) dtype=[('AOcol', '<i4', (2,)), ('BFcol', '<i4', (2,)), ('BFgam', '<i4', (2,)), ('CIcol', '<i4', (2,)), ('CMgam', '<i4', (2,)), ('FRgam', '<i4', (2,)), ('GAgam', '<i4', (2,)), ('GHcol', '<i4', (2,)), ('GHgam', '<i4', (2,)), ('GM', '<i4', (2,)), ('GNcol', '<i4', (2,)), ('GNgam', '<i4', (2,)), ('GQgam', '<i4', (2,)), ('GW', '<i4', (2,)), ('KE', '<i4', (2,)), ('UGgam', '<i4', (2,))]
   nbytes=63.8M cbytes=5.4M cratio=11.8
   values=allel.chunked.storage_zarr.ZarrTable>

In [7]:
ac_chri_3R_first = allel.AlleleCountsChunkedArray(calldata_out['3R']['chri']).compress(loc1)
ac_chri_3R_first

<AlleleCountsChunkedArray shape=(522469, 2) dtype=int32 chunks=(65309, 2)
   nbytes=4.0M cbytes=381.7K cratio=10.7
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

----------------------------------------
## Taking out the second region

In [8]:
loc = pos_all.intersect_range(2920000, 37000000)
loc1, loc2 = pos_all.locate_intersection(loc)

In [9]:
ac_3R_second = count_alleles_pos('3R', 2920000, 37000000)
ac_3R_second

<AlleleCountsChunkedTable shape=(7731390,) dtype=[('AOcol', '<i4', (2,)), ('BFcol', '<i4', (2,)), ('BFgam', '<i4', (2,)), ('CIcol', '<i4', (2,)), ('CMgam', '<i4', (2,)), ('FRgam', '<i4', (2,)), ('GAgam', '<i4', (2,)), ('GHcol', '<i4', (2,)), ('GHgam', '<i4', (2,)), ('GM', '<i4', (2,)), ('GNcol', '<i4', (2,)), ('GNgam', '<i4', (2,)), ('GQgam', '<i4', (2,)), ('GW', '<i4', (2,)), ('KE', '<i4', (2,)), ('UGgam', '<i4', (2,))]
   nbytes=943.8M cbytes=85.5M cratio=11.0
   values=allel.chunked.storage_zarr.ZarrTable>

In [10]:
ac_chri_3R_second = allel.AlleleCountsChunkedArray(calldata_out['3R']['chri']).compress(loc1)
ac_chri_3R_second

<AlleleCountsChunkedArray shape=(7731390, 2) dtype=int32 chunks=(120803, 2)
   nbytes=59.0M cbytes=5.4M cratio=11.0
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

-----------------------------------
## Merge the datasets

In [34]:
ac_3R_first.stack(ac_3R_second)

AttributeError: 'ZarrTable' object has no attribute 'stack'

In [23]:
allel.AlleleCountsChunkedArray(ac_3R_gste_out)

TypeError: bad number of dimensions: expected 2; found 1

In [ ]:
ac_3R_chri_gste_out = np.vstack(ac_chri_3R_first, ac_chri_3R_second)
ac_3R_chri_gste_out

-------------------------------------------------------
## Perform the ABBA-BABA test
Using the old script

In [7]:
chromosomes = ['3R']

In [8]:
ac_cache = dict()

def load_ac(chrom, pop):
    if (chrom, pop) in ac_cache:
        return ac_cache[chrom, pop]
    else:
        if pop in out_species:
            ac = calldata_out[chrom][pop]
        else:
            ac = calldata_biallel[chrom][pop]
            ac_cache[chrom, pop] = ac
        return ac
    
fourpop_cache = dict()

In [9]:
def f4_analysis(chroms, A, B, C, D, regions=None, blen=100000, plot=False, ax=None):
    
    
    
    region_str = ",".join(["{0}_{1}".format(r.start, r.stop) for r in regions])
    
    key = (",".join(chroms), region_str, A, B, C, D, blen)
    if key in fourpop_cache:
        # re-use from cache
        d, d_se, d_z, d_vb, d_vj = fourpop_cache[key]
        
    else:
        # load allele counts
        aca = np.vstack([load_ac(chrom, A)[region] for chrom, region in zip(chroms, regions)])
        acb = np.vstack([load_ac(chrom, B)[region] for chrom, region in zip(chroms, regions)])
        acc = np.vstack([load_ac(chrom, C)[region] for chrom, region in zip(chroms, regions)])
        acd = np.vstack([load_ac(chrom, D)[region] for chrom, region in zip(chroms, regions)])
            
        # run D test
        d, d_se, d_z, d_vb, d_vj = allel.blockwise_patterson_d(aca, acb, acc, acd, 
                                                                     blen=blen)
        
        # cache for re-use
        fourpop_cache[key] = d, d_se, d_z, d_vb, d_vj
    
    return d, d_se, d_z, d_vb, d_vj

def f4_analys(As, Bs, Cs, Ds, chroms=chromosomes, regions=None, blen=100000):
    
    if regions is None:
        regions = [slice(None)] * 4
    
    region_str = ",".join(["{0}_{1}".format(r.start, r.stop) for r in regions])
    # normalise inputs
    if not isinstance(chroms, (list, tuple)):
        chroms = [chroms]
    if not isinstance(As, (list, tuple)):
        As = [As]
    if not isinstance(Bs, (list, tuple)):
        Bs = [Bs]
    if not isinstance(Cs, (list, tuple)):
        Cs = [Cs]
    if not isinstance(Ds, (list, tuple)):
        Ds = [Ds]
        
    # setup output table
    tbl = [['chromosome', 'test', 'D', 'SE', 'Z']]

    for A in As:
        for B in Bs:
            if A != B:
                for C in Cs:
                    for D in Ds:
                        if C != D:
                            d, d_se, d_z, _, _ = f4_analysis(chroms, A, B, C, D, 
                                                             regions=regions, blen=blen)
                            test = 'D(%s, %s; %s, %s)' % (A, B, C, D)
                            row = [",".join(chroms), test, d, d_se, d_z]
                            tbl += [row]

    # display results
    (etl
     .wrap(tbl)
     .interpolate('D', '%.3f')
     .interpolate('SE', '%.4f')
     .interpolate('Z', '%.1f')
     .displayall(index_header=False, 
                 tr_style=lambda row: 'background-color: %s' % ('#afa' if float(row.Z) > 5 else '#aaf' if float(row.Z) < -5 else 'white')))

In [38]:
A = an_col
B = an_na
C = an_gam
D = 'chri'

In [39]:
f4_analys(A, B, C, D, chroms=("3R"),
                regions=(slice(1000000, 2820000), slice(2920000, 37000000)))

chromosome,test,D,SE,Z
3R,"D(AOcol, GM; GHgam, chri)",0.046,0.0047,9.8
3R,"D(AOcol, GM; CMgam, chri)",0.048,0.0042,11.3
3R,"D(AOcol, GM; BFgam, chri)",0.047,0.0042,11.3
3R,"D(AOcol, GM; GNgam, chri)",0.047,0.0043,10.9
3R,"D(AOcol, GM; GQgam, chri)",0.058,0.0041,14.1
3R,"D(AOcol, GM; UGgam, chri)",0.050,0.0045,11.1
3R,"D(AOcol, GM; GAgam, chri)",0.056,0.0051,11.0
3R,"D(AOcol, GM; FRgam, chri)",0.051,0.0048,10.5
3R,"D(AOcol, GW; GHgam, chri)",0.049,0.0047,10.4
3R,"D(AOcol, GW; CMgam, chri)",0.051,0.0043,11.7


In [42]:
A = an_col
B = an_gam
C = 'KE'
D = 'chri'

In [43]:
f4_analys(A, B, C, D, chroms=("3R"),
                regions=(slice(1000000, 2820000), slice(2920000, 37000000)))

chromosome,test,D,SE,Z
3R,"D(AOcol, GHgam; KE, chri)",0.038,0.0053,7.1
3R,"D(AOcol, CMgam; KE, chri)",0.037,0.0043,8.6
3R,"D(AOcol, BFgam; KE, chri)",0.037,0.0042,8.8
3R,"D(AOcol, GNgam; KE, chri)",0.037,0.0042,8.8
3R,"D(AOcol, GQgam; KE, chri)",0.030,0.0055,5.5
3R,"D(AOcol, UGgam; KE, chri)",0.028,0.0036,7.6
3R,"D(AOcol, GAgam; KE, chri)",0.019,0.0054,3.6
3R,"D(AOcol, FRgam; KE, chri)",-0.048,0.0146,-3.3
3R,"D(BFcol, GHgam; KE, chri)",0.008,0.0036,2.2
3R,"D(BFcol, CMgam; KE, chri)",0.007,0.0021,3.4


------------

In [10]:
A = an_col
B = an_gam
C = 'KE'
D = 'mela'

In [11]:
f4_analys(A, B, C, D, chroms=("3R"),
                regions=(slice(1000000, 2820000), slice(2920000, 37000000)))

chromosome,test,D,SE,Z
3R,"D(AOcol, GHgam; KE, mela)",0.053,0.0057,9.3
3R,"D(AOcol, CMgam; KE, mela)",0.052,0.0049,10.7
3R,"D(AOcol, BFgam; KE, mela)",0.052,0.0049,10.7
3R,"D(AOcol, GNgam; KE, mela)",0.051,0.0045,11.3
3R,"D(AOcol, GQgam; KE, mela)",0.046,0.0062,7.4
3R,"D(AOcol, UGgam; KE, mela)",0.040,0.0043,9.3
3R,"D(AOcol, GAgam; KE, mela)",0.027,0.0060,4.5
3R,"D(AOcol, FRgam; KE, mela)",-0.056,0.0136,-4.1
3R,"D(BFcol, GHgam; KE, mela)",0.013,0.0045,2.8
3R,"D(BFcol, CMgam; KE, mela)",0.012,0.0033,3.5


In [12]:
A = an_col
B = an_gam
C = 'KE'
D = 'meru'

In [13]:
f4_analys(A, B, C, D, chroms=("3R"),
                regions=(slice(1000000, 2820000), slice(2920000, 37000000)))

chromosome,test,D,SE,Z
3R,"D(AOcol, GHgam; KE, meru)",0.057,0.0055,10.5
3R,"D(AOcol, CMgam; KE, meru)",0.056,0.0049,11.6
3R,"D(AOcol, BFgam; KE, meru)",0.057,0.0050,11.3
3R,"D(AOcol, GNgam; KE, meru)",0.056,0.0048,11.8
3R,"D(AOcol, GQgam; KE, meru)",0.049,0.0065,7.5
3R,"D(AOcol, UGgam; KE, meru)",0.044,0.0045,9.7
3R,"D(AOcol, GAgam; KE, meru)",0.033,0.0059,5.6
3R,"D(AOcol, FRgam; KE, meru)",-0.044,0.0142,-3.1
3R,"D(BFcol, GHgam; KE, meru)",0.012,0.0044,2.8
3R,"D(BFcol, CMgam; KE, meru)",0.012,0.0036,3.2
